<a href="https://colab.research.google.com/github/arsb29/web-scraping/blob/main/%D0%92%D0%B5%D0%B1_%D1%81%D0%BA%D1%80%D0%B0%D0%BF%D0%B8%D0%BD%D0%B3_%D1%81_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%D0%BC_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
from bs4 import BeautifulSoup

In [2]:
def get_article_info(article, supplement_to_article):
  title = article.find('a', {'class': 'tm-article-snippet__title-link'}).span.get_text()
  datetime = article.find('span', {'class': 'tm-article-snippet__datetime-published'}).time['datetime']
  author = article.find('a', {'class': 'tm-user-info__userpic'})['title']

  id = article['id']
  script_data = supplement_to_article[id]
  description = BeautifulSoup(script_data['leadData']['textHtml'], 'html.parser').get_text()
  tags = [i['titleHtml'] for i in script_data['tags']]
  link = f"https://habr.com/ru/post/{id}/"

  article_result = {
    'title': title,
    'datetime': datetime,
    'author': author,
    'description': description,
    'tags': tags,
    'link': link
  }

  return article_result

In [3]:
def habr_parse(query, pages):
  results = {
    'query': query,
    'articles': []
  }
  pages_range = [str(i) for i in range(1, pages + 1)]
  for page in pages_range:
    data = requests.get(f'https://habr.com/ru/search/page{page}/?q={query}')
    if data.status_code == 200:
      bs = BeautifulSoup(data.text, 'html.parser')
      articles = bs.find_all('article', {'class': 'tm-articles-list__item'})
      script = bs.find_all('script')[4].get_text();
      supplement_to_article = json.loads(script[script.find('=') + 1: script.find(';(')])['articlesList']['articlesList']
      for article in articles:
        results['articles'].append(get_article_info(article, supplement_to_article))

  return results

In [4]:
QUERY = 'web3'
PAGES = 2

In [5]:
result = habr_parse(QUERY, PAGES)

In [6]:
with open('habr_parse.json', 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False)